# Fusion of ResNet34
* Check best 3 vs 3 vs 6

In [2]:
import os
import sys
import glob
import shutil
import random
import pickle
import numpy as np
from PIL import Image
import time
import copy
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.autograd import Variable
import torchvision.transforms as transforms
import torch.utils.data
from torchvision.models import resnet34
from torchvision import datasets, models, transforms
import torch.optim as optim
from torch.optim import lr_scheduler


# Add the src directory for functions
src_dir = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), 'src')
print(src_dir)
sys.path.append(src_dir)

# import my functions:
from functions import*

# Set it to use GPU1
torch.cuda.set_device(0)
print(torch.cuda.is_available())
print(torch.cuda.current_device())

/media/rene/Data/idc/src
True
0


In [3]:
def make_fusion_model(model_list, model_name_list, save_path, input_size, model=WeightedSum):
    model_list_ft = []    
    for idx, model_arch in enumerate(model_list):
        # get the proper model architecture
        model_ft = model_arch(pretrained=False)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, 2)
        model_ft = model_ft.cuda()
        # load the saved weights
        model_ft.load_state_dict(torch.load(os.path.join(save_path, model_name_list[idx])))
        model_list_ft.append(model_ft)
        
    fusion_model = model(num_input=6)
    fusion_model = fusion_model.cuda()
    return model_list_ft, fusion_model

### Fusion with best 3 models

In [4]:
model_list = [resnet34, resnet34, resnet34]
model_name_list = ['resnet34_0', 'resnet34_5', 'resnet34_2']    

batch_size = 16
num_workers = 6
save_path = '/media/rene/Data/data/idc/models'
PATH = '/media/rene/Data/data/idc'
input_size = 6

dataloaders, dataset_sizes = make_batch_gen(PATH, batch_size, num_workers, valid_name='valid', size=197)
model_list_ft, fusion_model = make_fusion_model(model_list, model_name_list, input_size, model=WeightedSum)

In [5]:
num_epochs = 6
criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.Adam(fusion_model.parameters(), lr=0.001)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=3, gamma=0.1)

model = train_fusion_model(fusion_model, model_list_ft, 
                    criterion, optimizer_ft, exp_lr_scheduler, num_epochs,
                     dataloaders, dataset_sizes)

torch.save(model.state_dict(), os.path.join(save_path, 'w_sum_fusion_3resnet34_b3'))

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 0/9
----------


/home/rene/miniconda3/envs/fastai/lib/python3.6/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)


train Loss: 0.2739 Acc: 0.8839


 10%|█         | 1/10 [34:46<5:12:55, 2086.16s/it]

...

valid Loss: 0.3292 Acc: 0.8648
saving model with acc  0.8648298061447549
Epoch 3/9
----------
train Loss: 0.2661 Acc: 0.8887


 40%|████      | 4/10 [2:19:04<3:28:37, 2086.24s/it]

valid Loss: 0.3246 Acc: 0.8644
Epoch 4/9
----------


Process Process-50:
Process Process-54:
Process Process-52:
Process Process-51:
Process Process-49:
Process Process-53:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/rene/miniconda3/envs/fastai/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/rene/miniconda3/envs/fastai/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/rene/miniconda3/envs/fastai/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/rene/miniconda3/envs/fastai/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/rene/miniconda3/envs/fastai/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/home/rene/minicond

KeyboardInterrupt: 

In [ ]:
dataloaders, dataset_sizes= make_batch_gen(PATH, batch_size, num_workers, 
                                                        valid_name='valid', test_name='test', size=197)

model_list_ft, fusion_model = make_fusion_model(model_list, model_name_list, input_size, model=WeightedSum)
fusion_model.load_state_dict(torch.load(os.path.join(save_path, 'weighted_sum_fusion_3resnet34_nosamp')))
batch_size = 8
num_workers = 6
criterion = nn.CrossEntropyLoss()

print('Validation weighted_sum_fusion_3resnet34_scratch_nosamp: ')
eval_fusion_model(fusion_model, model_list_ft, dataloaders['valid'], dataset_sizes['valid'], criterion)

print('Test weighted_sum_fusion_3resnet34_scratch_nosamp: ')
eval_fusion_model(fusion_model, model_list_ft, dataloaders['test'], dataset_sizes['test'], criterion)

In [ ]:
## Averaging???